In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import Imputer
import numpy as np
from PreprocessingKeyWords import preprocessing
preprocessor=preprocessing()

In [7]:
train_data=pd.read_csv("trainData.csv")

#Data Preprocessing

#removing the hmid
train_data = train_data.drop(['hmid','concepts'], axis=1)

#remove missing values
train_data=train_data.replace(('prefer not to say'),np.NAN)
#print(train_data.isnull().sum())
train_data=train_data.dropna()

#print(train_data.isnull().sum())
# print(len(train_data))

#creating features for moments data
happycorpus=preprocessor.getSocialKeywords(train_data['moment'])


from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
vectorizer=TfidfVectorizer(analyzer='word',min_df = 1, stop_words = 'english', sublinear_tf=True)
tfidf_X=vectorizer.fit_transform(happycorpus)

In [8]:
y_agency = train_data.agency
y_social=train_data.social
print("Step 1:",type(y_social))

#train n test data split
train_X, test_X, train_y, test_y = train_test_split(tfidf_X.toarray(), y_social, test_size=0.25)
print("done !!")

# train_X, test_X, train_y, test_y = train_test_split(tfidf_X.toarray(), y_agency, test_size=0.25)
# print("done !!")


Step 1: <class 'pandas.core.series.Series'>
done !!


In [9]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
model = XGBClassifier()
model.fit(train_X, train_y)
print(model)
# make predictions for test data
y_pred = model.predict(test_X)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)


In [11]:
print(y_pred)
predictions = [value for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(test_y, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

['no' 'yes' 'no' ... 'no' 'no' 'no']
Accuracy: 81.95%
